# Transfer Learning with ResNet (PyTorch)

## Objective
The goal of this notebook is to apply **transfer learning** using a pretrained **ResNet-18** model in PyTorch to perform image classification on the CIFAR-10 dataset.

Specifically, we will:
- Load a ResNet model pretrained on ImageNet
- Freeze the convolutional backbone to reuse learned visual features
- Replace the final fully connected layer for a new classification task
- Train only the classifier head for efficient learning
- Evaluate performance on a validation dataset

This approach reflects **industry-standard practice** for computer vision tasks such as medical imaging, satellite imagery, and real-world object recognition, where labeled data is often limited.

In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [52]:
train_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [54]:
train_dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    transform=train_transforms,
    download=True
)

val_dataset = datasets.CIFAR10(
    root="./data",
    train=False,
    transform=val_transforms,
    download=True
)

train_dataset = torch.utils.data.Subset(train_dataset, range(5000))
val_dataset = torch.utils.data.Subset(val_dataset, range(1000))

train_loader = DataLoader(train_dataset , batch_size=16, shuffle=True,num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False,num_workers=4)

class_names = train_dataset.dataset.classes

Files already downloaded and verified
Files already downloaded and verified


In [55]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

In [56]:
for param in model.parameters():
    param.requires_grad = False

In [57]:
num_classes = 10
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)



In [58]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

In [59]:
def train_epoch(model, loader):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

    return running_loss / len(loader), correct / total

In [60]:
def validate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = outputs.max(1)
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)

    return correct / total

sum(p.requires_grad for p in model.parameters())

2

In [61]:
epochs = 5

for epoch in range(epochs):
    train_loss, train_acc = train_epoch(model, train_loader)
    val_acc = validate(model, val_loader)

    print(
        f"Epoch {epoch+1}/{epochs} | "
        f"Loss: {train_loss:.4f} | "
        f"Train Acc: {train_acc:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )

Epoch 1/5 | Loss: 1.2051 | Train Acc: 0.6072 | Val Acc: 0.7370
Epoch 2/5 | Loss: 0.7938 | Train Acc: 0.7334 | Val Acc: 0.7590
Epoch 3/5 | Loss: 0.7500 | Train Acc: 0.7462 | Val Acc: 0.7670
Epoch 4/5 | Loss: 0.7046 | Train Acc: 0.7630 | Val Acc: 0.7460
Epoch 5/5 | Loss: 0.6867 | Train Acc: 0.7668 | Val Acc: 0.7570


In [62]:
torch.save(model.state_dict(), "resnet18_transfer_cifar10.pth")

## Fine-Tuning the Last ResNet Block

In this section, we fine-tune the final convolutional block (`layer4`) of ResNet-18.
This allows the model to adapt higher-level features to the CIFAR-10 dataset while
keeping lower-level features frozen to avoid overfitting and reduce training cost.

In [ ]:
for param in model.layer4.parameters():
    param.requires_grad = True

sum(p.requires_grad for p in model.parameters())